In [ ]:
# Does not need to be executed if ~/.ipython/profile_default/ipython_config.py exists and contains:
# get_config().InteractiveShell.ast_node_interactivity = 'all'

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

A number of functions to essentially, get as input two strings of digits,
$\sigma$ and $\tau$, and compute natural numbers $a$ and $b$ such that $a / b$
is reduced and equal to $0.\sigma\tau\tau\tau\tau\ldots$

# Computation of the fraction in nonreduced form

\begin{split}
0.\sigma\tau\tau\tau\dots & = \sigma 10^{-|\sigma|} + \tau(10^{-|\sigma|-|\tau|}
                         + 10^{-|\sigma|-2|\tau|} + 10^{-|\sigma|-3|\tau|} + \dots)\\
          & = \sigma 10^{-|\sigma|} + \frac{\tau 10^{-|\sigma|-|\tau|}}{(1 - 10^{-|\tau|})}\\
          & = \sigma 10^{-|\sigma|} + \frac{\tau 10^{-|\sigma|}}{(10^{|\tau|} - 1)}\\
          & = \frac{\sigma 10^{-|\sigma|} (10^{|\tau|} - 1) + \tau 10^{-|\sigma|}}{(10^{|\tau|} - 1)}\\
          & = \frac{\sigma (10^{|\tau|} - 1) + \tau}{(10^{|\tau|} - 1) 10^{|\sigma|}}
\end{split}

# Computation of the gcd of $a$ and $b$ with $a\geq 0$ and $b>0$

Since $a = (a\div b)b + a\bmod b$:
* if $n$ divides both $a$ and $b$ then it divides both $a$ and $(a\div b)b$, hence it divides $a - (a\div b)b$, hence it divides $a\bmod b$;
* conversely, if $n$ divides both $b$ and $a\bmod b$ then it divides $(a \div b)b + a\bmod b$, hence it divides $a$.

Hence $n$ divides both $a$ and $b$ iff $n$ divides both $b$ and $a\bmod b$. So $\gcd(a,b)=\gcd(b,a\bmod b)$.

Since $a\bmod b<b$, we get a sequence of equalities of the form:
$\gcd(a,b)=\gcd(a_1,b_1)=\gcd(a_2,b_2)=\dots=\gcd(a_{k-1},b_{k-1})=\gcd(a_k,0)$
with $k\geq 1$ and $b>b_1>b_2>\dots>b_{k-1}>0$; as $\gcd(a_k,0)=a_k$, $a_k$ is the gcd of $a$ and $b$.

In [ ]:
def gcd(a, b):
    while b:
        print(a, b)
        a, b = b, a % b
    print(a, b)
    
for a, b in (1233, 1233), (1233, 990), (990, 1233):
    print(f'\nTracing the computation of gcd of {a} and {b}:')
    gcd(a, b)

# Tuple versus tuple's elements

In [ ]:
def f(a, b):
    return 2 * a, 2 * b

try:
    f((1, 3))
except TypeError:
    'f() expects two arguments, not one!'
f(1, 3)
f(*f(1, 3)) # f(f(1, 3)) would be f((2, 6)); f(*f(1, 3)) is f(2, 6)
f(*f(*f(1, 3)))

In [ ]:
# x is the tuple of all arguments passed to f().
def f(*x):
    return x * 2

f()
f(0)
f(*f(0))
f(*f(*f(0)))

In [ ]:
# Written by Eric Martin for COMP9021


def determine_reduced_fraction_from_pattern_and_repeated_pattern():
    '''
    Gets as input two strings of digits, sigma and tau,
    and computes natural numbers a and b such that
    a / b is reduced and equal to 0.(sigma)(tau)(tau)(tau)(tau)...
    '''
    output_result(*determine_reduced_fraction(*get_both_inputs()))

def determine_reduced_fraction(sigma, tau):
    '''   
    >>> determine_reduced_fraction('0', '0')
    (0, 1)
    >>> determine_reduced_fraction('0', '1')
    (1, 90)
    >>> determine_reduced_fraction('1', '0')
    (1, 10)
    >>> determine_reduced_fraction('1', '1')
    (1, 9)
    >>> determine_reduced_fraction('9', '9')
    (1, 1)
    >>> determine_reduced_fraction('23', '905')
    (11941, 49950)
    >>> determine_reduced_fraction('1', '234')
    (137, 1110)
    >>> determine_reduced_fraction('234', '1')
    (2107, 9000)
    >>> determine_reduced_fraction('000', '97')
    (97, 99000)
    >>> determine_reduced_fraction('97', '000')
    (97, 100)
    >>> determine_reduced_fraction('01234', '543210')
    (51439249, 4166662500)
    '''
    return reduce_fraction(*compute_fraction(sigma, tau))
   
def compute_fraction(sigma, tau):
    '''
    Based on the computation
            0.(sigma)(tau)(tau)(tau)...
          = sigma * 10^{-|sigma|} + tau(10^{-|sigma|-|tau|} +
                                        10^{-|sigma|-2|tau|} +
                                        ...)
          = sigma * 10^{-|sigma|} +
            tau * 10^{-|sigma|-|tau|} / (1 - 10^{-|tau|})
          = sigma * 10^{-|sigma|} +
            tau * 10^{-|sigma|} / (10^{|tau|} - 1)
          = [sigma * 10^{-|sigma|} * (10^{|tau|} - 1) + tau * 10^{-|sigma|}] /
            (10^{|tau|} - 1)
          = [sigma * (10^{|tau|} - 1) + tau] /
            [(10^{|tau|} - 1) * 10^{|sigma|}]
                                  
    >>> compute_fraction('0', '0')
    (0, 90)
    >>> compute_fraction('0', '1')
    (1, 90)
    >>> compute_fraction('1', '0')
    (9, 90)
    >>> compute_fraction('1', '1')
    (10, 90)
    >>> compute_fraction('9', '9')
    (90, 90)
    >>> compute_fraction('23', '905')
    (23882, 99900)
    >>> compute_fraction('1', '234')
    (1233, 9990)
    >>> compute_fraction('234', '1')
    (2107, 9000)
    >>> compute_fraction('000', '97')
    (97, 99000)
    >>> compute_fraction('97', '000')
    (96903, 99900)
    >>> compute_fraction('01234', '543210')
    (1234541976, 99999900000)
    '''
    numerator = int(sigma) * (10 ** len(tau) - 1) + int(tau)
    denominator = (10 ** len(tau) - 1) * 10 ** len(sigma)
    return numerator, denominator

def reduce_fraction(numerator, denominator):
    '''
    >>> reduce_fraction(0, 1)
    (0, 1)
    >>> reduce_fraction(1, 90)
    (1, 90)
    >>> reduce_fraction(9, 90)
    (1, 10)
    >>> reduce_fraction(10, 90)
    (1, 9)
    >>> reduce_fraction(90, 90)
    (1, 1)
    >>> reduce_fraction(23882, 99900)
    (11941, 49950)
    >>> reduce_fraction(1233, 9990)
    (137, 1110)
    >>> reduce_fraction(2107, 9000)
    (2107, 9000)
    >>> reduce_fraction(97, 99000)
    (97, 99000)
    >>> reduce_fraction(96903, 99900)
    (97, 100)
    >>> reduce_fraction(1234541976, 99999900000)
    (51439249, 4166662500)
    '''
    if numerator == 0:
        return 0, 1
    the_gcd = gcd(numerator, denominator)
    return numerator // the_gcd, denominator // the_gcd

# Just to practice; in practice, we would just import gcd from the math module.
def gcd(a, b):
    '''
    Euclid's algorithm

    >>> gcd(0, 1)
    1
    >>> gcd(1, 90)
    1
    >>> gcd(9, 90)
    9
    >>> gcd(10, 90)
    10
    >>> gcd(90, 90)
    90
    >>> gcd(23882, 99900)
    2
    >>> gcd(1233, 990)
    9
    >>> gcd(2107, 9000)
    1
    >>> gcd(97, 9000)
    1
    >>> gcd(96903, 99900)
    999
    >>> gcd(1234541976, 99999900000)
    24
    '''
    while b:
        a, b = b, a % b
    return a

def get_both_inputs():
    print('We want to compute the reduced fraction, a / b,\n   that evaluates to '
                                                                        '.(sigma)(tau)(tau)(tau)...'
         )
    return get('sigma'), get('tau')

def get(sigma_or_tau):
    while True:
        value = input(f'Input {sigma_or_tau}: ')
        if value.isnumeric():
            return value
        print('Incorrect input, try again.')

def output_result(numerator, denominator):
    print(f'The fraction is: {numerator} / {denominator}')
    print(f'It evaluates to: {numerator / denominator}')


if __name__ == '__main__':
    import doctest
    doctest.testmod()